In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name())

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [1]:
from transformers import BertForSequenceClassification, BertTokenizer, BertModel

c:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

class Bert(nn.Module):
    def __init__(self, product_classes):
        super(Bert, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.food_classifier = nn.Linear(self.bert.config.hidden_size, product_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Use the [CLS] token representation
        hazard_logits = self.hazard_classifier(pooled_output)
        return hazard_logits
        

In [ ]:
class FoodHazardDataset(Dataset):
    def __init__(self, texts, hazard_labels, product_labels, tokenizer, max_length = 128):
        self.texts = texts
        self.hazard_labels = hazard_labels
        self.product_labels = product_labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def len(self):
        return len(self.texts)
    
    def getItem(self, index):
        text = self.texts[index]
        hazard_label = self.hazard_labels[index]
        product_label = self.product_labels[index]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'hazard_label': torch.tensor(hazard_label, dtype=torch.long),
            'product_label': torch.tensor(product_label, dtype=torch.long)
        }
         